In [26]:
import pandas as pd
import ast

Definimos una funcion para evaluar que tipo de datos devuelva la clave name dentro del diccionario y filtrar los valores float

In [27]:
def get_collection_name(x):
    if pd.isnull(x):
        return None
    try:
        collection = ast.literal_eval(x)
        return collection['name']
    except (SyntaxError, KeyError, TypeError, ValueError):
        return None

In [ ]:
# Cargar el dataset original
df = pd.read_csv('movies_dataset.csv')

# Desanidar campos anidados
df['belongs_to_collection'] = df['belongs_to_collection'].astype(str)
df['belongs_to_collection'] = df['belongs_to_collection'].apply(get_collection_name)
df['production_companies'] = df['production_companies'].apply(lambda x: eval(x)[0]['name'] if (pd.notnull(x) and eval(x)) else None)

# Rellenar valores nulos con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Eliminar filas con valores nulos en release_date
df = df.dropna(subset=['release_date'])

# Formatear fechas y extraer el año de estreno
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df['release_year'] = df['release_date'].dt.year

# Convertir las columnas a numericas
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# eliminamos los 'NaN'
df = df.dropna(subset=['revenue', 'budget'])

# Calcular retorno de inversión
df['return'] = df['revenue'] / df['budget']
df['return'] = df['return'].fillna(0)

# Eliminar columnas no utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage']
df = df.drop(columns=columns_to_drop)



# Guardar el dataset transformado
df.to_csv('dataset_transformed.csv', index=False)